In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np
import cupy as cp
import shutil
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_dir = "/content/drive/MyDrive/RAVDESS/FacialTracking_Actors_01-24"
output_dir = "/content/drive/MyDrive/RAVDESS/audio"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# import librosa
# import librosa.display
# audio_path="/content/drive/MyDrive/RAVDESS/audio/03-01-01-01-01-01-01.wav"
# relative_ldmrk_path="/content/drive/MyDrive/RAVDESS/FacialTracking_Actors_01-24/01-01-01-01-01-01-01.csv"
# relative_ldmrk_df=pd.read_csv(relative_ldmrk_path)
# fps=30
# y,sr=librosa.load(audio_path,sr=None)
# hop_length=int(sr/fps)
# mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=hop_length)
# mfcc = mfcc[:, :relative_ldmrk_df.shape[0]]

# print(f"MFCC shape: {mfcc.shape}")  # (13, num_frames)

MFCC shape: (13, 98)


In [ ]:
# import librosa
# import librosa.display
# audio_path="/content/drive/MyDrive/RAVDESS/audio"
# ldmrk_path="/content/drive/MyDrive/RAVDESS/Landmarks_Processed"
# fps=30
# emotion_map={
#     '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad', '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
# }

# statement_map = {
#     "01": "Kids are talking by the door",
#     "02": "Dogs are sitting by the door"
# }

# intensity_map={
#     '01': 'normal', '02': 'strong'
# }

# for filename in os.listdir(audio_path):
#   if filename.endswith(".wav") and (filename.startswith("03-") or filename.startswith("02-")):
#     audio_file=os.path.join(audio_path,filename)
#     ldmrk_filename="01"+filename[2:].replace(".wav",".pkl")
#     ldmrk_file=os.path.join(ldmrk_path,ldmrk_filename)
#     if not os.path.exists(ldmrk_file):
#       print(f"{filename} not found")
#       continue
#     y,sr=librosa.load(audio_file,sr=None)
#     hop_length=int(sr/fps)
#     mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40, hop_length=hop_length) #---use n_mfcc=13 if only 13 features are required xxxx
#     mfcc = mfcc[:, :ldmrk_df.shape[0]]

#     with open(ldmrk_file, "rb") as f:
#       ldmrk_points = pickle.load(f)
#     num_frames = len(ldmrk_points)

#     if mfcc.shape[0] < num_frames:
#       pad_width = num_frames - mfcc.shape[0]
#       mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
#     elif mfcc.shape[0] > num_frames:
#       mfcc = mfcc[:num_frames, :]

#     combined_data = [{'landmarks': np.array(ldmrk_points[i]), 'mfcc': mfcc[i]} for i in range(num_frames)]
#     save_file = os.path.join(save_path, pkl_filename)
#     with open(save_file, "wb") as f:
#       pickle.dump(combined_data, f)

#     name_parts=filename.replace('.wav','').split("-")
#     _, _, emotion_code, intensity, statement_code, _, _=name_parts
#     labels.append({
#             "filename": pkl_filename,
#             "emotion_label": emotion_map.get(emo_code,"unknown"),
#             "emotion_intensity": intensity_map.get(intensity,"unknown"),
#             "statement_text": statement_map.get(statement_code,"unknown")
#     })

#     print(f"✅ Processed and saved {pkl_filename}")

# import pandas as pd
# label_df = pd.DataFrame(labels)
# label_df.to_csv(os.path.join(save_path, "emotion_labels.csv"), index=False)
# print(f"\n🎯 Done! Labels saved at {save_path}/emotion_labels.csv")

Updated 01-01-01-01-01-01-01.csv with MFCCs
All landmark CSVs updated with MFCCs and metadata columns.


In [3]:
import os
import numpy as np
import pickle
import pandas as pd
import librosa

audio_path = "/content/drive/MyDrive/RAVDESS/audio"
selective_ldmrk_path = "/content/drive/MyDrive/RAVDESS/Landmarks_Processed"
save_path = "/content/drive/MyDrive/RAVDESS/Landmarks_WithAudio"
os.makedirs(save_path, exist_ok=True)

fps = 30
emotion_map = {'01':'neutral','02':'calm','03':'happy','04':'sad','05':'angry','06':'fearful','07':'disgust','08':'surprised'}
statement_map = {"01":"Kids are talking by the door","02":"Dogs are sitting by the door"}
intensity_map = {'01':'normal','02':'strong'}

labels = []

for filename in os.listdir(audio_path):
    if filename.endswith(".wav") and (filename.startswith("03-") or filename.startswith("02-")):
        audio_file = os.path.join(audio_path, filename)
        pkl_filename = "01" + filename[2:].replace(".wav", ".pkl")
        ldmrk_file = os.path.join(selective_ldmrk_path, pkl_filename)
        if not os.path.exists(ldmrk_file):
            print(f"{filename} not found")
            continue

        # --- Load audio and MFCC ---
        y, sr = librosa.load(audio_file, sr=None)
        hop_length = int(sr / fps)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40, hop_length=hop_length).T  # (num_frames, 40)

        # --- Load landmarks ---
        with open(ldmrk_file, "rb") as f:
            ldmrk_points = pickle.load(f)  # list of frames: (num_frames, num_points, 2)
        num_frames = len(ldmrk_points)

        # --- Align MFCC to landmarks ---
        if mfcc.shape[0] < num_frames:
            pad_width = num_frames - mfcc.shape[0]
            mfcc = np.pad(mfcc, ((0, pad_width), (0,0)), mode='constant')
        elif mfcc.shape[0] > num_frames:
            mfcc = mfcc[:num_frames, :]

        # --- Combine landmarks + MFCC per frame ---
        combined_data = [{'landmarks': np.array(ldmrk_points[i]), 'mfcc': mfcc[i]} for i in range(num_frames)]

        # --- Save combined .pkl ---
        save_file = os.path.join(save_path, pkl_filename)
        with open(save_file, "wb") as f:
            pickle.dump(combined_data, f)

        # --- Label info ---
        name_parts = filename.replace('.wav','').split("-")
        _, _, emotion_code, intensity, statement_code, _, _ = name_parts
        labels.append({
            "filename": pkl_filename,
            "emotion_label": emotion_map.get(emotion_code,"unknown"),
            "emotion_intensity": intensity_map.get(intensity,"unknown"),
            "statement_text": statement_map.get(statement_code,"unknown")
        })

        print(f"✅ Processed and saved {pkl_filename}")

# --- Save labels ---
labels_path="/content/drive/MyDrive/RAVDESS"
label_df = pd.DataFrame(labels)
label_df.to_csv(os.path.join(labels_path, "emotion_labels.csv"), index=False)
print(f"\n🎯 Done! Labels saved at {labels_path}/emotion_labels.csv")


✅ Processed and saved 01-01-01-01-01-01-01.pkl
✅ Processed and saved 01-02-03-01-02-02-01.pkl
✅ Processed and saved 01-02-04-01-02-02-01.pkl
✅ Processed and saved 01-02-03-02-01-01-01.pkl
✅ Processed and saved 01-02-03-01-01-01-01.pkl
✅ Processed and saved 01-02-02-02-01-02-01.pkl
✅ Processed and saved 01-02-05-01-01-02-01.pkl
✅ Processed and saved 01-02-03-01-01-02-01.pkl
✅ Processed and saved 01-02-02-02-02-01-01.pkl
✅ Processed and saved 01-02-04-01-01-02-01.pkl
✅ Processed and saved 01-02-01-01-01-01-01.pkl
✅ Processed and saved 01-02-04-02-02-01-01.pkl
✅ Processed and saved 01-02-03-02-02-01-01.pkl
✅ Processed and saved 01-02-05-01-01-01-01.pkl
✅ Processed and saved 01-02-05-01-02-01-01.pkl
✅ Processed and saved 01-02-05-02-02-01-01.pkl
✅ Processed and saved 01-02-03-02-01-02-01.pkl
✅ Processed and saved 01-02-02-01-02-01-01.pkl
✅ Processed and saved 01-02-02-01-01-01-01.pkl
✅ Processed and saved 01-02-04-02-01-01-01.pkl
✅ Processed and saved 01-02-04-02-02-02-01.pkl
✅ Processed a

In [8]:
import os

path = "/content/drive/MyDrive/RAVDESS/Landmarks_WithAudio"
num_pkl_files = len([f for f in os.listdir(path) if f.endswith('.csv')])
print(f"Number of .pkl files: {num_pkl_files}")


Number of .pkl files: 0
